In [ ]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
# berechnet einen Schritt mit dem Expliziten Eulerverfahren
def euler(y, t, dt, f):
    return y + dt * f(t, y)

# berechnet einen Schritt mit dem Verfahren von Heun
def heun(y, t, dt, f):
    f_eval = f(t, y)
    y_ = y + dt * f_eval
    return y + 0.5 * dt * (f_eval + f(t + dt, y_))

# berechnet einen Schritt mit dem klassischen Runge-Kutta Verfahren
def runge_4(y, t, dt, f):
    T_1 = f(t, y)
    T_2 = f(t + 0.5 * dt, y + 0.5 * dt * T_1)
    T_3 = f(t + 0.5 * dt, y + 0.5 * dt * T_2)
    T_4 = f(t + dt, y + dt * T_3)
    return y + dt / 6. * (T_1 + 2 * T_2 + 2 * T_3 + T_4)

# berechnet die Nährerungslösung für y(t) mit einem beliebigen Einschrittverfahren
def zeitintegration(y_0, t, f, verfahren):
    n = t.shape[0]
    
    y = np.zeros((y_0.shape[0], n))
    y[:,0] = y_0
    for i in range(0, n-1):
        dt = t[i+1] - t[i]
        y[:,i+1] = verfahren(y[:,i], t[i], dt, f)
    return y

In [ ]:
def f(t, y):
    # f = m * a
    # a = dv / dt
    # v = dh / dt
    # Zweidimensionales System mit Ort und Geschwindigkeit als Unbekannte:
    # dh / dt = v
    # dv / dt = a = f / m
    mass_stone = 1.0 #kg
    diameter_stone = 0.01 # m^2
    
    height = y[0]
    velocity = y[1]
    
    # Stein ist am Boden aufgekommen
    # Ort und Geschwindigkeit sind beide 0
    if height <= 0:
        return np.array([0.0, 0.0])
        
    # Gravitationsgesetz, https://de.wikipedia.org/wiki/Newtonsches_Gravitationsgesetz
    mass_earth = 5.9722e24 #kg, https://de.wikipedia.org/wiki/Erdmasse
    gravity_constant = 6.67430e-11 # m^3 / (kg * s^2), https://de.wikipedia.org/wiki/Gravitationskonstante
    earth_radius = 6378137 #m, https://de.wikipedia.org/wiki/Erdradius
    distance = earth_radius + height
    gravity = gravity_constant * mass_stone * mass_earth / earth_radius**2
    # Luftreibung, https://de.wikipedia.org/wiki/Str%C3%B6mungswiderstand
    c_w = 0.45 # no unit, https://de.wikipedia.org/wiki/Str%C3%B6mungswiderstandskoeffizient
    # https://wind-data.ch/tools/luftdichte.php
    density_air = 1.247015 * np.exp(-height *  0.000104)
    friction = 0.5 * c_w * diameter_stone * density_air * velocity**2
    
    # Reibung wirkt entgegen der Geschwindigkeit
    force = -np.sign(velocity) * friction
    # Gravitation wirkt immer nach unten
    force -= gravity
    
    return np.array([velocity, force / mass_stone])

In [ ]:
# Anzahl der Intervalle
n = 400
t = np.linspace(0,50,n+1,endpoint=True)

# Starte in 2000m Höhe, mit 20m/s vertikaler Geschwindigkeit
y_0 = np.array([1000, 20])
y_runge_4 = zeitintegration(y_0, t, f, runge_4)
    
plt.plot(t, y_runge_4[0,:], label="Höhe")
plt.plot(t, y_runge_4[1,:], label="Geschwindigkeit")
plt.xlabel("Zeit (s)")
plt.ylabel("Höhe (m), Geschwindigkeit (m/s)")
plt.legend()
plt.show()